# Imports

In [3]:
import fasttext.util
from fasttext import load_model as fasttext_load_model

from sklearn.preprocessing import normalize
from sklearn.metrics.pairwise import cosine_similarity

import numpy as np
import pandas as pd

from google.colab import drive

In [4]:
drive.mount('/content/drive')

Mounted at /content/drive


# Loading the pretrianed model

We didn't use the `fasttext.util.download` method since it was super slow due to unknown reasons. Instead we're going to manually download the pretrained model checkpoints and extract it.

In [1]:
!wget "https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.fa.300.bin.gz" -O "/content/cc.fa.300.bin.gz"

--2024-02-29 15:26:48--  https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.fa.300.bin.gz
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 52.84.251.106, 52.84.251.114, 52.84.251.15, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|52.84.251.106|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4502524724 (4.2G) [application/octet-stream]
Saving to: ‘/content/cc.fa.300.bin.gz’

/content/cc.fa.300. 100%[===================>]   4.19G  21.2MB/s    in 4m 12s  

2024-02-29 15:31:00 (17.0 MB/s) - ‘/content/cc.fa.300.bin.gz’ saved [4502524724/4502524724]



In [5]:
!gzip -dv "/content/cc.fa.300.bin.gz"

/content/cc.fa.300.bin.gz:	 37.9% -- replaced with /content/cc.fa.300.bin


In [6]:
ft_fa = fasttext_load_model('/content/cc.fa.300.bin')

# Building the corpus

In [7]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/NLP/Project/Dataset/final_dataset.csv')

In [8]:
df['total_summary'] = df.apply(lambda row: '\n'.join([item for item in [row['summary_digi'],row['summary_kio'],row['summary_tiny'],row['summary_uptv'],row['story']] if str(item) != 'nan']), axis=1)

In [9]:
with open('/content/farsi_corpus.txt', 'w') as f:
  f.write('\n'.join(df['total_summary'].to_list()))

In [10]:
!head "/content/farsi_corpus.txt"

داستان مدیر یک آژانس تبلیغاتی به نام شان، و اوون، یک صنعتگر و نوازنده شهری کوچک، که خانه های خود را با شروع شدن تعطیلات، با یکدیگر عوض می کنند.
این مستند موضوعات متعددی را بررسی می کند که به هم پیوسته اند و یکی از تأثیرگذارترین گروه ها در راک اند رول را گرد هم آورده اند.
داستان یک دختر نوجوان هندی-آمریکایی که دچار بحران هویتی فرهنگی است و با دوست صمیمی‌اش نیز به مشکل خورده، اما در همین میان تصادفاً یک موجود شیطانی را آزاد می‌کند که از احساس تنهایی او تغذیه می‌کند و رشد می‌کند.
یک نوجوان هندی-آمریکایی که با چالش های هویت فرهنگی خود دست و پنجه نرم می کند، با بهترین دوست سابق خود درگیر می شود و در این روند، ناخواسته موجودی شیطانی را آزاد می کند که با تغذیه از تنهایی او قوی تر می شود.
یک نوجوان هندی-آمریکایی که با هویت فرهنگی خود دست و پنجه نرم می کند، با بهترین دوست سابق خود درگیر می شود و در این روند، ناخواسته موجودی شیطانی را آزاد می کند که با تغذیه از تنهایی او قوی تر می شود.
فیلم در درون زندگی می کند It Lives Inside محصول کشور آمریکا و در ژانر ترسناک ، درام می‌باشد و به کارگردانی Bish

# Training `FASTTEXT`

In [11]:
from fasttext.FastText import read_args, _build_args, unsupervised_default
import fasttext_pybind as fasttext

def train_unsupervised(*kargs, **kwargs):
    ft = kwargs['ft']
    kwargs.pop('ft', None)

    arg_names = ['input', 'model', 'lr', 'dim', 'ws', 'epoch', 'minCount',
                  'minCountLabel', 'minn', 'maxn', 'neg', 'wordNgrams', 'loss', 'bucket',
                  'thread', 'lrUpdateRate', 't', 'label', 'verbose', 'pretrainedVectors']
    args, manually_set_args = read_args(kargs, kwargs, arg_names, unsupervised_default)
    a = _build_args(args, manually_set_args)

    ft.set_args(args=a)

    fasttext.train(ft.f, a)
    ft.set_args(ft.f.getArgs())

    return ft

In [12]:
ft_fa = train_unsupervised('/content/farsi_corpus.txt', ft=ft_fa)

In [17]:
ft_fa.save_model('/content/drive/MyDrive/Colab Notebooks/NLP/Project/ft.cc.fa.300.finetuned.bin')

In [15]:
!gzip -k /content/ft.cc.fa.300.finetuned.bin

In [16]:
!cp "/content/ft.cc.fa.300.finetuned.bin.gz" "/content/drive/MyDrive/Colab Notebooks/NLP/Project/"